- change seed in tabnet model to (0,100,2)
- import clipping

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold
from torch.nn.modules.loss import _WeightedLoss

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
#X = train[g_feats+c_feats].copy().values
#select = VarianceThreshold(threshold=0.7)
#X_new = select.fit_transform(X)
#drop_feats = list(np.array(train[g_feats+c_feats].columns)[select.get_support()==False])
#len(drop_feats)

#train.drop(drop_feats, axis=1, inplace=True)
#test.drop(drop_feats, axis=1, inplace=True)

#g_feats = [i for i in train.columns if "g-" in i]
#c_feats = [i for i in train.columns if "c-" in i]

# feature engineering

In [7]:
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [8]:
c_num = 1
pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
pca = PCA(n_components=c_num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

g_num = 10
pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
pca = PCA(n_components=g_num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [9]:
#uc_num = 1
#um = UMAP(n_neighbors=30, n_components=uc_num, random_state=42)
#um_c_cols = ["um-c"+str(i+1) for i in range(uc_num)]#
#uc_train = um.fit_transform(train[c_feats])
#uc_test = um.transform(test[c_feats])
#uc_train = pd.DataFrame(uc_train, columns=um_c_cols)
#uc_test = pd.DataFrame(uc_test, columns=um_c_cols)

#ug_num = 10
#um = UMAP(n_neighbors=30, n_components=ug_num, random_state=42)
#um_g_cols = ["um-g"+str(i+1) for i in range(ug_num)]
#ug_train = um.fit_transform(train[g_feats])
#ug_test = um.transform(test[g_feats])
#ug_train = pd.DataFrame(ug_train, columns=um_g_cols)
#ug_test = pd.DataFrame(ug_test, columns=um_g_cols)

#train = pd.concat([train, uc_train],axis=1)
#test = pd.concat([test, uc_test],axis=1)
#train = pd.concat([train, ug_train],axis=1)
#test = pd.concat([test, ug_test],axis=1)

In [10]:
#n_clusters_g = 15
#n_clusters_c = 5
#def create_cluster(train, test, kind, n_clusters):
#    if kind == "g":
#        train_ = train[pca_g_cols].copy()
#        test_ = test[pca_g_cols].copy()
#    else:
#        train_ = train[pca_c_cols].copy()
#        test_ = test[pca_c_cols].copy()    
#    kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(train_)
#    train[f'clusters_{kind}'] = kmeans.labels_
#    test[f'clusters_{kind}'] = kmeans.predict(test_)
#    train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#    test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#    return train, test
    
#train, test = create_cluster(train, test, kind = 'g', n_clusters = n_clusters_g)
#train, test = create_cluster(train, test, kind = 'c', n_clusters = n_clusters_c)

In [11]:
def fe(df):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})      
    tmp.drop(["sig_id", "cp_type"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 885) (3982, 885)


In [12]:
train.head()

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,pca-g1,pca-g2,pca-g3,pca-g4,pca-g5,pca-g6,pca-g7,pca-g8,pca-g9,pca-g10
0,24,0,1.111801,0.903367,-0.433829,-0.971728,-0.286559,-1.011388,-1.357431,-0.041716,...,-5.947541,-6.565323,7.922011,-7.719018,4.405187,1.806038,3.393864,1.992503,2.330853,0.229401
1,72,0,0.105667,0.672509,0.257486,0.086759,1.199685,0.691813,0.353695,0.558374,...,-5.162087,-0.264433,-12.554034,4.953071,0.799825,-0.025325,1.039128,-0.897690,3.815383,3.735017
2,48,0,0.767036,0.942499,1.408911,-0.126492,-0.028694,1.490985,0.272541,0.359490,...,0.677536,8.687788,-2.526892,0.283035,0.538570,2.956883,-2.141152,3.346174,-1.128757,-4.715069
3,48,0,-0.755626,-0.297077,-0.455058,0.765972,2.343522,-0.852713,-2.316440,0.301512,...,10.869688,10.066734,-0.212951,-5.016873,-7.041268,-3.342125,-3.466734,6.218932,-4.034827,0.560658
4,72,1,-0.468806,-0.504196,0.956769,0.975864,1.447729,-0.863807,-0.346926,-0.227072,...,-6.933980,6.061535,-8.831261,-4.482883,-7.548974,-8.808425,-2.480037,-4.121178,-1.091505,-1.924720


In [13]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()
    
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [14]:
fn_train.shape, fn_test.shape, fn_targets.shape

((21948, 885), (3982, 885), (21948, 206))

# modelling

In [15]:
class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):

        logits = 1 / (1 + np.exp(-y_pred))
        
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [16]:
MAX_EPOCH=200
device = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def modelling_tabnet(tr, target, te, sample_seed):
    seed_everything(sample_seed) 
    tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.5, seed = sample_seed,
                     lambda_sparse=0, n_independent=1, n_shared=1, 
                     optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []

    NB_SPLITS = 5
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = np.zeros([len(tr),target.shape[1]])
    scores = []
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
    
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=512, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits) 
    
        preds_val = model.predict(X_val)
        # Apply sigmoid to the predictions
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        oof_preds[val_idx,:] = preds
        scores.append(score)

        # preds on test
        preds_test = model.predict(te)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))
        
    test_preds_all = np.stack(test_cv_preds)
    print("OOF log loss:", log_loss(np.ravel(target), np.ravel(np.array(oof_preds))))
    aucs = []
    for task_id in range(206):
        aucs.append(roc_auc_score(y_true=target[:, task_id],y_score=oof_preds[:, task_id]))
    print(f"Overall AUC : {np.mean(aucs)}")
    return oof_preds, test_preds_all

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [0,100,2]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(fn_train, fn_targets, fn_test, seed_)
    target_oof += oof_preds / len(seeds)
    target_pred += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 0.24635 | val_logits_ll: 0.02777 |  0:00:02s
epoch 10 | loss: 0.01777 | val_logits_ll: 0.02051 |  0:00:13s
epoch 20 | loss: 0.01691 | val_logits_ll: 0.0174  |  0:00:26s
epoch 30 | loss: 0.01644 | val_logits_ll: 0.01686 |  0:00:37s
epoch 40 | loss: 0.01612 | val_logits_ll: 0.01661 |  0:00:48s
epoch 50 | loss: 0.016   | val_logits_ll: 0.01694 |  0:00:59s
epoch 60 | loss: 0.01583 | val_logits_ll: 0.0166  |  0:01:11s
epoch 70 | loss: 0.01544 | val_logits_ll: 0.01661 |  0:01:22s
epoch 80 | loss: 0.01532 | val_logits_ll: 0.01684 |  0:01:34s
epoch 90 | loss: 0.01494 | val_logits_ll: 0.01654 |  0:01:46s

Early stopping occured at epoch 97 with best_epoch = 77 and best_val_logits_ll = 0.01651
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 0.24705 | val_logits_ll: 0.02829 |  0:00:01s
epoch 10 | loss: 0.01776 | val_logits_ll: 0.01893 |  0:00:12s
epoch 20 | loss: 0.01677 | val_logits_ll: 0.01766 |  

In [17]:
aucs = []
for task_id in range(targets.shape[1]-1):
    aucs.append(roc_auc_score(y_true=targets.iloc[:, task_id+1].values,
                              y_score=target_oof[:, task_id]))
print(f"Overall AUC : {np.mean(aucs)}")

Overall AUC : 0.777312424402112


In [18]:
p_min = 0.001
p_max = 1 - p_min

t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.clip(np.array(train_checkscore.iloc[:,1:]),p_min, p_max))))

OOF log loss:  0.01488423472199443
OOF log loss:  0.015138063540444575


In [19]:
sub[target_feats] = np.clip(target_pred, p_min, p_max)
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)